In [ ]:
import pandas as pd 
import tensorflow as tf
import random
from tensorflow.keras.models import load_model

In [ ]:
TEST_PATH = "../input/petfinder-pawpularity-score/train.csv"
TEST_IMAGE_FOLDER = "../input/petfinder-pawpularity-score/train/"

#TEST_PATH = "../input/petfinder-pawpularity-score/test.csv"
#TEST_IMAGE_FOLDER = "../input/petfinder-pawpularity-score/test/"

AUTOTUNE = tf.data.experimental.AUTOTUNE  
img_size = 224
channels = 3
Batch_size = 64

In [ ]:
df = pd.read_csv(TEST_PATH)
df["Address"] = df["Id"].apply(lambda x : TEST_IMAGE_FOLDER + x + ".jpg")

In [ ]:
df = df[["Id", "Address"]]

In [ ]:
def image_preprocess():  
    def augment(image):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_saturation(image, 0.95, 1.05)
        image = tf.image.random_contrast(image, 0.95, 1.05)
        return image
    def can_be_augmented(img, label):
        return augment(img), label    
    return augment



def image_read():
    def decode(path):
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=channels)
        image = tf.cast(image, tf.float32)
        image = tf.image.resize(image, (img_size, img_size))
        return image
    def can_be_decoded(path, label):
        return decode(path), label
    return decode


def create_dataset(df, batch_size, augment = False, shuffle = False):
    image_read_fn = image_read()
    image_preprocess_fn = image_preprocess()
    dataset = tf.data.Dataset.from_tensor_slices((df["Address"].values))
    dataset = dataset.map(image_read_fn, num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(image_preprocess_fn, num_parallel_calls=AUTOTUNE) if augment else dataset
    dataset = dataset.shuffle(1024, reshuffle_each_iteration=True) if shuffle else dataset
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
modelAll = load_model('../input/modelall/preTrainedModelAll')

In [ ]:
predAll = modelAll.predict(create_dataset(df, Batch_size, augment = True, shuffle = True))

In [ ]:
L = []
for j in range(len(predAll)):
    L += [ int( predAll[j]*100. ) ]

In [ ]:
array_id = list(df["Id"])
dict_all = { array_id[j]: L[j] for j in range(len(L)) }

In [ ]:
pre_df_out = [ (id_index, dict_all[id_index]) for id_index in  array_id ]

In [ ]:
df_out = pd.DataFrame( pre_df_out, columns=["Id", "Pawpularity"] )

In [ ]:
df_out.to_csv('submission.csv', index=False)

In [ ]:
df_out